# AI Census

AI Project, which predicts whether income exceeds $50K/yr based on census data. Also known as "Census Income" dataset.

## Data Exploration

The data exploration phase involves analyzing the dataset to understand the distribution of the variables, identifying any missing or outliers, and determining which features are relevant for the prediction phase.

### Importing the libraries

In [6]:
import pandas as pd

### Importing the dataset

In [26]:
column_names = ["age", "workclass", "fnlwgt", "education", "education-num", "marital-status", "occupation", "relationship", "race", "sex", "capital-gain", "capital-loss", "hours-per-week", "native-country", "salary"]

df_census = pd.read_csv('input/adult.data', names=column_names, index_col=False)

df_census.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [20]:
df_census.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32561 entries, 39 to 52
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  object
 1   workclass       32561 non-null  int64 
 2   fnlwgt          32561 non-null  object
 3   education       32561 non-null  int64 
 4   education-num   32561 non-null  object
 5   marital-status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  int64 
 10  capital-gain    32561 non-null  int64 
 11  capital-loss    32561 non-null  int64 
 12  hours-per-week  32561 non-null  object
 13  native-country  32561 non-null  object
dtypes: int64(5), object(9)
memory usage: 3.7+ MB


In [14]:
df_census.describe()

,39,77516,13,2174,0,40
count,32560.000000,3.256000e+04,32560.000000,32560.000000,32560.000000,32560.000000
mean,38.581634,1.897818e+05,10.080590,1077.615172,87.306511,40.437469
std,13.640642,1.055498e+05,2.572709,7385.402999,402.966116,12.347618
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178315e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783630e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370545e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


In [19]:
df_census['education'].value_counts()

9     10501
10     7291
13     5355
14     1723
11     1382
7      1175
12     1067
6       933
4       646
15      576
5       514
8       433
16      413
3       333
2       168
1        51
Name: education, dtype: int64

## Data Visualization

Data visualization helps to understand the relationship between different variables and identify patterns in the data.
It allows to detect outliers and trends that might not be apparent from a simple statistical summary.

## Data Preprocessing

Data preprocessing is the process of cleaning and formatting the data to make it suitable for training the model.
This might include filling in missing values, normalizing the data, and encoding categorical variables.

## Training

The training phase is where the model is trained to fit to the data provided by the dataset, using a set of parameters called hyperparameters.
The goal is to find the optimal set of hyperparameters that will produce the best model for the prediction task.
But you also have to be careful about an overfitting or underfitting model.

## Hyperparameter Tuning

Hyperparameter tuning is the process of fine-tuning the model's hyperparameters to improve its performance on the prediction task.

## Evaluation

The evaluation phase involves assessing the performance of the model on a set of unknown realistic data to the model.
This process is usually done by comparing the predicted values to the true values, and calculating metrics such as accuracy, precision, and recall.